<a href="https://colab.research.google.com/github/ss1705/ai-traffic-system/blob/main/TrafficLLM/Few_Shot_Learning_Approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers accelerate torch sentencepiece --quiet

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

In [ ]:
import torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    load_in_8bit=True,  # Quantized model to save RAM
    device_map="auto"  # Uses GPU efficiently
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

##Basic Prompting

In [ ]:
prompt = """
Act as a traffic flow prediction expert. Predict future traffic volume based on historical data.

### Example 1:
**Input:**
- Location: Downtown, New York, USA
- Weather: Cloudy, Temperature: 10°C
- Time: Monday, 8 AM
- Past Traffic Volumes (last 6 hours): [340, 320, 280, 260, 250, 230]
**Prediction:** [220, 200, 180, 160, 140, 120]

### Example 2:
**Input:**
- Location: Silicon Valley, California, USA
- Weather: Sunny, Temperature: 25°C
- Time: Saturday, 2 PM
- Past Traffic Volumes (last 6 hours): [150, 170, 190, 210, 230, 250]
**Prediction:** [270, 290, 310, 330, 350, 370]

### New Input:
**Input:**
- Location: District 3 in Yolo, California, USA
- Weather: Sunny, Temperature: 9.5°C
- Time: Monday, 12 AM
- Past Traffic Volumes (last 6 hours): [196, 208, 204, 174, 215, 176]

Make the prediction. Output the next 6 traffic volumes as an array. Also provide an explanation for the prediction made.
"""

In [ ]:
import transformers

In [ ]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer # Adjust based on available memory
)

Device set to use cuda:0


In [ ]:
output = pipeline(
    prompt,
    max_new_tokens=150,  # Stops output early
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    eos_token_id=tokenizer.eos_token_id
)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
print(output[0]['generated_text'])


Act as a traffic flow prediction expert. Predict future traffic volume based on historical data.

### Example 1:
**Input:**
- Location: Downtown, New York, USA
- Weather: Cloudy, Temperature: 10°C
- Time: Monday, 8 AM
- Past Traffic Volumes (last 6 hours): [340, 320, 280, 260, 250, 230]
**Prediction:** [220, 200, 180, 160, 140, 120]

### Example 2:
**Input:**
- Location: Silicon Valley, California, USA
- Weather: Sunny, Temperature: 25°C
- Time: Saturday, 2 PM
- Past Traffic Volumes (last 6 hours): [150, 170, 190, 210, 230, 250]
**Prediction:** [270, 290, 310, 330, 350, 370]

### New Input:
**Input:**
- Location: District 3 in Yolo, California, USA
- Weather: Sunny, Temperature: 9.5°C
- Time: Monday, 12 AM
- Past Traffic Volumes (last 6 hours): [196, 208, 204, 174, 215, 176]

Make the prediction. Output the next 6 traffic volumes as an array. Also provide an explanation for the prediction made.

### Example Output:
**Output:** [190, 180, 170, 160, 150, 140]

Explanation: The predictio